## Import Libraries

In [30]:
# !pip install pyconll

In [1]:
from conllu import parse_incr
import os
import sys
import re
import pyconll
import tensorflow as tf 

2025-05-08 18:04:54.511539: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-08 18:04:54.839628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746716694.994300  341748 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746716695.050519  341748 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746716695.310580  341748 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## login huggingface hub

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


## Load Data

In [3]:
# with open("Arabic_POS.conllu", "r", encoding="utf-8") as f:
#     for i in range(10):
#         for tokenlist in parse_incr(f):
#             print(tokenlist)

with open("Arabic_POS.conllu", "r", encoding="utf-8") as f:
    count = 0
    for tokenlist in parse_incr(f):
        print(tokenlist)
        count += 1
        if count == 10:
            break


TokenList<برلين, ترفض, حصول, شركة, اميركية, على, رخصة, تصنيع, دبابة, ", ليوبارد, ", الالمانية, metadata={newdoc id: "afp.20000715.0075", newpar id: "afp.20000715.0075:p1", sent_id: "afp.20000715.0075:p1u1", text: "برلين ترفض حصول شركة اميركية على رخصة تصنيع دبابة "ليوبارد" الالمانية"}>
TokenList<برلين, 15, -, 7, (, اف, ب, ), -, افادت, صحيفة, الاحد, الالمانية, ", ويلت, ام, سونتاغ, ", في, عددها, عدد, ها, الصادر, غدا, ،, ان, المستشار, غيرهارد, شرودر, يرفض, حصول, المجموعة, الاميركية, ", جنرال, ديناميكس, ", على, رخصة, لتصنيع, ل, تصنيع, الدبابة, الالمانية, ", ليوبارد, 2, ", عبر, شراء, المجموعة, الحكومية, الاسبانية, للاسلحة, ل, الأسلحة, ", سانتا, بربارة, ", ., metadata={newpar id: "afp.20000715.0075:p2", sent_id: "afp.20000715.0075:p2u1", text: "برلين 15-7 (اف ب) - افادت صحيفة الاحد الالمانية "ويلت ام سونتاغ" في عددها الصادر غدا، ان المستشار غيرهارد شرودر يرفض حصول المجموعة الاميركية "جنرال ديناميكس" على رخصة لتصنيع الدبابة الالمانية "ليوبارد 2" عبر شراء المجموعة الحكومية الاسبانية للاسلحة "س

In [4]:
data = pyconll.load_from_file('Arabic_POS.conllu')

In [5]:
sentences = []
labels = []
for sentence in data:
  sentences.append([token.form for token in sentence])
  labels.append([token.upos for token in sentence])

In [6]:
unique_labels = set(label for sublist in labels for label in sublist if label is not None)
print(unique_labels)

{'SCONJ', 'SYM', 'PROPN', 'NOUN', 'VERB', 'AUX', 'ADP', 'PUNCT', 'X', 'PART', 'PRON', 'INTJ', 'ADV', 'NUM', 'CCONJ', 'DET', 'ADJ'}


In [7]:
labels = ['VERB', 'NOUN', 'X', 'CCONJ', 'SYM', 'PART', 'ADP', 'PUNCT', 'ADJ', 'DET', 'ADV', 'AUX', 'SCONJ', 'INTJ', 'PRON', 'PROPN', 'NUM']

In [8]:
sentences[1]

['برلين',
 '15',
 '-',
 '7',
 '(',
 'اف',
 'ب',
 ')',
 '-',
 'افادت',
 'صحيفة',
 'الاحد',
 'الالمانية',
 '"',
 'ويلت',
 'ام',
 'سونتاغ',
 '"',
 'في',
 'عددها',
 'عدد',
 'ها',
 'الصادر',
 'غدا',
 '،',
 'ان',
 'المستشار',
 'غيرهارد',
 'شرودر',
 'يرفض',
 'حصول',
 'المجموعة',
 'الاميركية',
 '"',
 'جنرال',
 'ديناميكس',
 '"',
 'على',
 'رخصة',
 'لتصنيع',
 'ل',
 'تصنيع',
 'الدبابة',
 'الالمانية',
 '"',
 'ليوبارد',
 '2',
 '"',
 'عبر',
 'شراء',
 'المجموعة',
 'الحكومية',
 'الاسبانية',
 'للاسلحة',
 'ل',
 'الأسلحة',
 '"',
 'سانتا',
 'بربارة',
 '"',
 '.']

## preprocessing the data 

In [9]:
import re

def preprocess_arabic_text(text):
    # Remove diacritics (optional but recommended)
    text = re.sub(r'[\u0610-\u061F]', '', text)

    # Remove unwanted punctuation (retain Arabic punctuation and other characters)
    # Keep Arabic characters, common punctuation, and spaces
    text = re.sub(r'[^\w\s\u0600-\u06FF،؛؟.!؟]', '', text)
    text = text.replace('،', '')

    # Remove extra whitespace (multiple spaces, etc.)
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove numbers (if needed)
    text = re.sub(r'\d+', '', text)

    # Remove non-Arabic characters (only keep Arabic characters and spaces)
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)

    # Remove text within parentheses, square brackets, and curly braces
    text = re.sub(r'\(.*?\)', '', text)  # Remove ()
    text = re.sub(r'\[.*?\]', '', text)  # Remove []
    text = re.sub(r'\{.*?\}', '', text)  # Remove {}

    # Remove text within angle brackets
    text = re.sub(r'<.*?>', '', text)  # Remove < >

    # Remove double or single quotes and punctuation marks
    text = re.sub(r'["“”\'‘’]', '', text)  # Remove " and '

    # Remove any leading or trailing whitespace
    text = text.strip()


    
    return text


In [10]:
data_preprocessed = []  
for sentence in sentences:
    preprocessed_sentence = []
    for word in sentence:
        preprocessed_word = preprocess_arabic_text(word)
        if preprocessed_word:  # Only add non-empty words
            preprocessed_sentence.append(preprocessed_word)
    data_preprocessed.append(preprocessed_sentence)

data_preprocessed[0:10]

[['برلين',
  'ترفض',
  'حصول',
  'شركة',
  'اميركية',
  'على',
  'رخصة',
  'تصنيع',
  'دبابة',
  'ليوبارد',
  'الالمانية'],
 ['برلين',
  'اف',
  'ب',
  'افادت',
  'صحيفة',
  'الاحد',
  'الالمانية',
  'ويلت',
  'ام',
  'سونتاغ',
  'في',
  'عددها',
  'عدد',
  'ها',
  'الصادر',
  'غدا',
  'ان',
  'المستشار',
  'غيرهارد',
  'شرودر',
  'يرفض',
  'حصول',
  'المجموعة',
  'الاميركية',
  'جنرال',
  'ديناميكس',
  'على',
  'رخصة',
  'لتصنيع',
  'ل',
  'تصنيع',
  'الدبابة',
  'الالمانية',
  'ليوبارد',
  'عبر',
  'شراء',
  'المجموعة',
  'الحكومية',
  'الاسبانية',
  'للاسلحة',
  'ل',
  'الأسلحة',
  'سانتا',
  'بربارة'],
 ['وفي',
  'و',
  'في',
  'نيسان',
  'ابريل',
  'الماضي',
  'تخلت',
  'الدولة',
  'الاسبانية',
  'عن',
  'مجموعة',
  'سانتا',
  'بربارة',
  'التي',
  'تصنع',
  'دبابات',
  'ليوبارد',
  'الالمانية',
  'الى',
  'جنرال',
  'ديناميكس',
  'التي',
  'تنتج',
  'الدبابة',
  'الاميركية',
  'ام',
  'ابرامس',
  'المعتبرة',
  'المنافسة',
  'الرئيسية',
  'لدبابة',
  'ل',
  'دبابة',
  'ليوبارد',
 

IndexError: string index out of range